In [1]:
import sys
sys.path.append('../scripts')

In [2]:
import pandas as pd
import spacy
import os
from pathlib import Path
from xml.dom import minidom
from charsplit import Splitter
from rules.ellipticCompounds import exchangeLemmas
from evaluation import error_analysis, get_scores
from dataset import load_data

nlp = spacy.load("de_core_news_lg")

In [3]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path=Path('..'), job_name='foo', version_base='1.1')
config = compose(config_name='experiment.yaml')

In [4]:
tmp_path = Path('../tmp/baseline_files')
tmp_path.mkdir(exist_ok=True, parents=True)

In [5]:
path_xml_file = tmp_path / 'Original.xml'
path_output_file = tmp_path / 'Output.xml'
path_gertwol_file = tmp_path / 'gertwol.txt'
path_word_freqs = tmp_path / 'wordFreqs.txt'
tagged_path = '../compoundListTagged.txt'

path_tokens = Path('..') / config.data.ggponc_plain_text

# Load the Data

In [6]:
base_path = Path('..')
_, _, test_df = load_data(base_path / config.data.cnf_tsv_path, base_path / config.data.controls_tsv_path)

In [7]:
# Get baseline results for test set
df = test_df.sort_values('file').reset_index()

In [8]:
len(df[['file', 'sentence_id']].drop_duplicates())

1159

In [9]:
df.head()

,index,file,sentence_id,text_whitespace_tokenized,resolved_whitespace_tokenized,raw_sentence,full_resolution,split,controls
0,0,00_mundhoehlenkarzinom_0002.tsv,1,chronischer Tabak - oder Alkoholabusus,chronischer Tabakabusus oder Alkoholabusus,Hauptrisikofaktoren für das Auftreten eines Mu...,Hauptrisikofaktoren für das Auftreten eines Mu...,test,False
1,1,00_mundhoehlenkarzinom_0002.tsv,2,chronischem Tabak - oder Alkoholabusus,chronischem Tabakabusus oder Alkoholabusus,Bei chronischem Tabak- oder Alkoholabusus ist ...,Bei chronischem Tabakabusus oder Alkoholabusus...,test,False
2,40,00_mundhoehlenkarzinom_0098.tsv,2,Radio - oder Radiochemotherapie,Radiotherapie oder Radiochemotherapie,Als kurativ intendierte therapeutische Optione...,Als kurativ intendierte therapeutische Optione...,test,False
3,44,00_mundhoehlenkarzinom_0103.tsv,1,guten Allgemein - und Leistungszustand,guten Allgemeinzustand und Leistungszustand,"Patienten mit einem unheilbaren Tumorleiden, j...","Patienten mit einem unheilbaren Tumorleiden, j...",test,False
4,281,00_mundhoehlenkarzinom_0115.tsv,30,NaN,NaN,Dies bestätigte sich auch nach 2 Jahren Nachbe...,Dies bestätigte sich auch nach 2 Jahren Nachbe...,test,True


# Optimal Values

In [10]:
ea = error_analysis(df.full_resolution.values, df.full_resolution.values, df.raw_sentence.values)
ea.error_type.value_counts()

tn    616
tp    543
Name: error_type, dtype: int64

In [11]:
get_scores(ea, "max")

{'max/exact_match': 1.0, 'max/gleu': 1.0, 'max/edit_distance_rel': 1.0}

# Trivial Baseline: Do Nothing

In [12]:
ea = error_analysis(df.raw_sentence.values, df.full_resolution.values, df.raw_sentence.values)
ea.error_type.value_counts() / len(ea)

tn    0.531493
fn    0.468507
Name: error_type, dtype: float64

In [13]:
get_scores(ea, "trivial")

{'trivial/exact_match': 0.5314926660914582,
 'trivial/gleu': 0.9472019923158214,
 'trivial/edit_distance_rel': 0.5314926660914582}

# Create XML Document

In [14]:
root = minidom.Document()

xml = root.createElement('corpus')
xml.setAttribute('id', 'corpus')
root.appendChild(xml)

for i, f in enumerate(df[['file', 'raw_sentence']].groupby('file').first().index.to_list()):
    file = root.createElement('file')
    file.setAttribute('n', str(i))
    xml.appendChild(file)

    div = root.createElement('div')
    file.appendChild(div)

    for j, s in enumerate(df[df['file']==f]['raw_sentence']):
        sentence = root.createElement('s')
        sentence.setAttribute('lang', 'de')
        sentence.setAttribute('n', f'{i}-{j}')
        div.appendChild(sentence)

        doc = nlp(s)
        for k, token in enumerate(doc):
            word = root.createElement('w')
            word.setAttribute('lemma', token.lemma_)
            word.setAttribute('n', f'{i}-{j}-{k}')
            word.setAttribute('pos', token.tag_)
            word.setAttribute('whitespace', token.whitespace_)
            sentence.appendChild(word)
            word.appendChild(root.createTextNode(token.text))

with open(path_xml_file, "wb") as f:
    f.write(root.toprettyxml(indent ="\t", encoding="utf-8"))

# Produce Gertwol-like List and Gather Word Frequencies

In [15]:
freq = {}
dct = {}
splitter = Splitter()
special_characters = "!@#$%^&*()-+?_=,<>/.[] "

for filename in os.listdir(path_tokens):
    with open(path_tokens / filename , encoding='utf8') as f:
        for word in f.readlines():
            word = word[:-1]

            # Gertwol Split
            split = splitter.split_compound(word)[0]
            if split[0] != 0:
                dct[word] = '#'.join([split[1]] + [subword.lower() for subword in split[2:]])

            # Word Freq
            if word in special_characters:
                continue
            if word in freq:
                freq[word] += 1
            else:
                freq[word] = 1

with open(path_gertwol_file, 'w', encoding='utf-8') as f:
    for key, value in dct.items():
        f.write(f'{key}\t{value}\n')

with open(path_word_freqs, 'w', encoding='utf-8') as f:
    for key, value in freq.items():
        f.write(f'{key}\t{value}\n')

# Run Aepli Baseline

In [16]:
exchangeLemmas(path_xml_file, path_output_file, path_gertwol_file, path_word_freqs, tagged_path)

Alkohol#abusus
Alkohol#abusus
Radio#chemotherapie
Leistungs#zustand
Leistungs#zustand
Leistungs#zustand
Gesamt#überleben
Proteom#ebene
Ref#lux
Dyspnö#symptomatik
Chemo#therapie
Ausschluss#kriterien
Post#operativen
Mort#alität
Tumor#bedingte
Schm#erzen
Strahlen#therapie
Strahlen#therapieinduzierten
Gemüse#reiche
Ern#ährung
Strahlen#therapie
Aroma#massage
Indikations#problem
Perforations#risiko
Papillen#adenome
Hnpcc#syndrom
Mr#kolonografie
Blutgefäß#invasion
Links#seitigen
Tum#oren
Tumor#lokalisation
Tonstein#industrie
Peri#kardkarzinose
Gefäßresektionsrän#der
Palliativ#beratung
Zweit-Generations#tki
OS
Zweit-Generations#tki
Zweiwöchentli#chen
Abstän#den
Stammzell#transplantation
Laser#therapie
Bewegungs#therapie
Panik#reaktion
Drittlinien#therapie
Panik#komponente
Hand#ventilatoren
Lebens#stunden
Langzeit#studie
60#jährigen
30#jährigen
65#jährigen
29#jährigen
Wiedervorstellung#algorithmen
Wiedervorstellung#algorithmen
Spät#komplikationen
Nach#teile
Therapie#optionen
Hilfs#angebote
Hilf

# Reconstruct Complete Sentences

In [17]:
dom = minidom.parse(open(path_output_file))
predictions = []

sentences = dom.getElementsByTagName('s')
for sentence in sentences:
    tokens = sentence.getElementsByTagName('w')
    sentence_tokens = []
    for token in tokens:
        word = token.childNodes[0].nodeValue
        lemma = token.getAttribute('lemma')
        if '+' in lemma:
            word_parts = lemma.partition('+')
            if token.getAttribute('pos') == 'TRUNC':
                word = word.replace('-', word_parts[2])
            else:
                word = word.replace('-', word_parts[0])
        whitespace = token.getAttribute('whitespace')
        sentence_tokens.append(word + whitespace)

    predictions.append(''.join(token for token in sentence_tokens))

# Evaluate Baseline

In [18]:
errors_df = error_analysis(predictions, df.full_resolution, df.raw_sentence)
errors_df.error_type.value_counts() / len(errors_df)

tn         0.530630
fn         0.195858
tp         0.170837
replace    0.043141
delete     0.031061
complex    0.018119
insert     0.009491
fp         0.000863
Name: error_type, dtype: float64

In [19]:
%%time
scores = get_scores(errors_df, "test")
scores

CPU times: user 1min 20s, sys: 75.1 ms, total: 1min 20s
Wall time: 1min 22s


{'test/exact_match': 0.7014667817083693,
 'test/gleu': 0.9647026057324476,
 'test/edit_distance_rel': 0.771210005623929}

In [20]:
from notebook_util import show_errors

In [22]:
errors = pd.concat([errors_df, df[['file', 'sentence_id']].reset_index()], axis=1)

In [23]:
show_errors(errors[(errors.error_type == 'replace')])

0;00_mundhoehlenkarzinom_0115.tsv;14

__Input:__

Diese Dreier-Kombination zeigte als erste überhaupt gegenüber Platin in Kombination mit 5-FU eine signifikant gesteigerte Ansprechrate, progressionsfreies- und Gesamtüberleben (10,1 vs 7,4 Monate mit einer HR von 0.8) und war somit in allen Effektivitätsparametern positiv.

__Error type:__ replace

1;01_magenkarzinom_0002.tsv;24

...

__Error type:__ replace

49;29_adulte-weichgewebesarkome_0381.tsv;13

__Input:__

Die VOYAGER Studie, eine Phase III Studie, evaluierte die Wirksamkeit und Sicherheit von Avapritinib vs. Regorafenib als Drittlinien- oder spätere Behandlung bei Patienten mit inoperablem oder metastasiertem GIST.

__Error type:__ replace